In [1]:
from src.core import SemanticCaching

In [2]:
cache = SemanticCaching()

/opt/homebrew/Caskroom/miniforge/base/envs/caching/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
question = 'How wold is the Nepal'
cache.ask(question, use_llm=True)

Generating answer using mistralai/Mistral-7B-v0.1
Cuda is not available


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score:  1.0
Found cache hit: 0aef613c-c878-4457-96c0-f6c4dcd87c14 with distance 1.0
Time to respond: 0.6648299694061279 seconds


'The answer to the question "How old is Nepal?" is that very little is known about the early history of Nepal, but legends and documented references reach far back to the 30th century BC. You can find more information on the history of Nepal on the Wikipedia page titled "History of Nepal."'

In [6]:
question = 'Who was the first prime minister of Nepal?'
cache.ask(question, use_llm=False)

Cuda is not available


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Score:  0.81710684
Found cache hit: e5921fd9-a572-472d-b983-2a0f74a4ffeb with distance 0.81710684
Time to respond: 1.477388858795166 seconds


'The first King of Nepal was Prithvi Narayan Shah.'

In [7]:
import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, pipeline)


def _get_answer_from_llm(data: str, base_model_id: str = ""):

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model_id, quantization_config=bnb_config)

    # left padding saves memory
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_id,
        model_max_length=4092,
        padding_side="left",
        add_eos_token=True)

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )

    sequences = pipe(
        data,
        do_sample=True,
        max_new_tokens=100,
        temperature=0.7,
        top_k=50,
        top_p=0.90,
        num_return_sequences=1
    )

    print(sequences[0]['generated_text'])
    return sequences[0]['generated_text']

In [8]:
_get_answer_from_llm(data="What is the currency of Nepal",
                     base_model_id="mistralai/Mistral-7B-v0.1")

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [3]:
from transformers import BitsAndBytesConfig

In [4]:
BitsAndBytesConfig(load_in_4bit=True,
                   bnb_4bit_quant_type="nf4",
                   bnb_4bit_use_double_quant=True,)

PackageNotFoundError: No package metadata was found for bitsandbytes